In [ ]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
# Load your CSV file using pandas
df = pd.read_csv('train.csv', encoding='ISO-8859-1')

# Create a custom dataset using the 'article' column
custom_dataset = Dataset.from_pandas(df)

# Tokenize the custom dataset
tokenizer = T5Tokenizer.from_pretrained('t5-large')



In [ ]:
def tokenize_function(example):
    inputs = tokenizer(example['article'], padding='max_length', truncation=True, return_tensors='pt')
    inputs['labels'] = inputs.input_ids.clone()
    return inputs

tokenized_dataset = custom_dataset.map(tokenize_function, batched=True)



In [ ]:
tokenized_dataset.save_to_disk("tokenized_dataset_directory")

In [1]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_from_disk

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:551: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
tokenized_dataset = load_from_disk("tokenized_dataset_directory")

In [3]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='Q3',          # Output directory for the model
    num_train_epochs=3,            # Number of training epochs
    per_device_train_batch_size=1, # Batch size per device during training
    save_steps=500,                # Save model every 500 steps
    save_total_limit=2,            # Only last 2 models are saved. Older ones are deleted.
)

In [4]:
# Load pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-large')

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [5]:
# Fine-tune the model
trainer.train()

Step,Training Loss
500,0.489100
1000,0.017800


TrainOutput(global_step=1497, training_loss=0.17251628450178352, metrics={'train_runtime': 13303.2966, 'train_samples_per_second': 0.113, 'train_steps_per_second': 0.113, 'total_flos': 3241075802112000.0, 'train_loss': 0.17251628450178352, 'epoch': 3.0})

In [7]:
trainer.save_model("fine_tuned_model_directory")

In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = 'Q3'  # Path to the directory where your trained model is saved
model = T5ForConditionalGeneration.from_pretrained("fine_tuned_model_directory")
tokenizer = T5Tokenizer.from_pretrained('t5-large')

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load your fine-tuned model and tokenizer
model_path = "fine_tuned_model_directory"
tokenizer = AutoTokenizer.from_pretrained('t5-large')
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Define a function to summarize a document without dividing it into chunks
def summarize_document(document):
    inputs = tokenizer.encode("summarize: " + document, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=50, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Load the CSV file with 'article' and 'summary' columns
input_csv_file = "test.csv"  # Replace with your input file path
output_csv_file = "output.csv"  # Replace with your desired output file name
data = pd.read_csv(input_csv_file)

# Process each row and generate summaries
generated_summaries = []

for index, row in data.iterrows():
    document = row['article']
    generated_summary = summarize_document(document)
    generated_summaries.append(generated_summary)

    # Print the generated summary for this row
    print(f"Generated Summary for Row {index + 1}:\n{generated_summary}\n")

# Create a new DataFrame with the generated summaries and actual summaries
output_data = pd.DataFrame({'generated_summary': generated_summaries, 'actual_summary': data['highlights']})

# Save the DataFrame to a new CSV file
output_data.to_csv(output_csv_file, index=False)


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Generated Summary for Row 1:
Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount

Generated Summary for Row 2:
Rahul Kumar, 17, clambered over the enclosure fence at the Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against

Generated Summary for Row 14:
warren Weinstein, who appears to have been the only American citizen held hostage by al Qaeda, was accidentally killed in a U.S. drone strike in January. But it didn't have to be that way. A senior U.S. official familiar with the handling of the issue told CNN that the U.S. government made no serious effort to negotiate for the 73-year-old development expert's release, either directly to al Qaeda or through proxies in Pakistan. Another senior U.S. official told CNN that Weinstein's capture by al Qaeda made it hard for the United States to negotiate, even though proxies such as the Pakistani government have links to intermediar

Generated Summary for Row 15:
Argentinian con-artist Sofia Davila posted raunchy pictures or herself online and then flirted with unsuspecting men she had contacted on the social network, suggesting they meet up for sex. But after meeting her victims, the 21-year-old from Buenos Aires, would spike their drinks and wait for them to f

Generated Summary for Row 27:
David Hyche, now a special agent at the Bureau of Alcohol, Tobacco and Firearms (ATF), developed the technology nine years ago when his four-month-old daughter turned blind. A transmitter is placed inside plastic eggs that let off a high-pitched beeping sound. Now Hyche has passed on the idea to the Alabama Institute for Deaf and Blind in Birmingham. Festive invention: Blind children in Birmingham, Alabama, have celebrated Easter early by hunting for plastic eggs that let of a beeping sound. They trade the eggs in for candy. The device: Bomb expert David Hyche put a transmitter in a plastic egg so his daughter could join the

Generated Summary for Row 28:
A newlywed husband who was stabbed to death in a hotel room had solicited his murderer during a Craigslist exchange for gay sex, according to new court records on Thursday. Jamyra Gallmon, 21, has been charged with first-degree murder while armed for allegedly killing David Messerschmitt, 30, on February 

Generated Summary for Row 40:
Mirko 'Cro Cop' Filipovic delivered on his promise to exact revenge as he stopped Gabriel Gonzaga in the third round of their long-awaited heavyweight rematch in Krakow. Eight years after their first encounter in which Gonzaga delivered a stunning knockout, Filipovic endured a tough opening two sessions before making his move. Mirko 'Cro Cop' Filipovic made a slow start against Gabriel Gonzaga. After dropping his rival to the canvas, he finished the job with his elbows before the refereee stepped in. Filipovic had not fought in the UFC since 2011 and made a slow start as Gonzaga stalk

Generated Summary for Row 41:
The comedian tweeted a picture on Thursday alongside former England captain David Beckham and New York Giants' NFL hotshot Odell Beckham Jnr. Accompanied with the caption: 'Double Beckhams!' Corden was all smiles as he posed in between the duo. James Corden (centre) posted a picture alongside Odell Beckham Jnr (left) and David Beckham via Twitte

...

Generated Summary for Row 456:
when the Washington couple's three-year-old had first began complaining of someone talking to him at night, his parents had put it down to their toddler's over-active imagination, reported CBS New York. That was until one night when they heard a strange man's voice come over the monitor to tell their son: 'Wake up little boy, daddy's looking for you.' A couple heard the terrifying message: 'Wake up little boy, daddy's looking for you' come through their 3-year-old son's baby monitor (stock image) And the horrified parents, who do not want to be named for fear the unknown man could track them down,

Generated Summary for Row 457:
Chinese villager who was desperate to become a grandfather has been arrested for buying a wife for his son, then reselling her to another family after realising she was infertile. The man, known only as Xu, even tried to impregnate the 'daughter-in-law' himself after discovering his son had not slept with his wife a single time. X

Generated Summary for Row 470:
Carl Hendrick, head of learning and research at Wellington College, Berkshire, has attacked the ‘tidal wave of guff’ in classrooms. These ‘missives in mediocrity’ often tell pupils to ‘live your dream’ and ‘you can do it’ and ‘be all that you can be’. Cheesy slogans such as ‘reach for the stars’ on posters in schools could be detrimental to pupils, it has been claimed. Above, a stock image of a motivational poster. But the glossy notices are ‘often reductively misinterpreted as “you can achieve anything if you believe”’. Mr Hendrick argues

Generated Summary for Row 471:
Sauntering down leafy avenues past typical Dutch step-­gabled buildings, carpets of crocuses, the occasional cyclist whooshing quietly by and the sun bouncing off the canal, I can think of few more pleasant places to spend a spring weekend than The Hague. This is also prime territory for a gallery­ lover like me: the beautiful, elegant and not-­too-­big Mauritshuis was restored last year.

Generated Summary for Row 483:
Alaa Abdullah Esayed admitted encouraging terrorism by posting 45,000 tweets supporting ISIS. A woman posted tens of thousands of Tweets supporting ISIS which included pictures of dead bodies and encouraging children to arm themselves with weapons. Alaa Abdullah Esayed, from south London, uploaded 45,600 tweets in less than a year. Some of the messages she posted between June 1 2013 and May 14 2014, included pictures of the corpses of fighters killed in Syria which she uploaded to her Instagram account. Her tweets also included a poem 'Mother of the Martyr' which advises parents how to raise a child to be violent with weapons

Generated Summary for Row 484:
From the best night's kip being enjoyed on a Wednesday to rising the earliest on a Sunday, new data is giving fascinating insights into how the world sleeps. Using statistics from Sleep Cycle app users, researchers have discovered that the earliest wake-up time worldwide is on a Monday in South Africa.

Generated Summary for Row 496:
Chelsea match-winner Cesc Fabregas hailed a vital win after his late goal gave his side a 1-0 victory over QPR at Loftus Road. The game looked to be heading for a stalemate before Fabregas' 88th minute shot - their first on target in the game - found its way past Rob Green. The win moved Jose Mourinho's men seven points clear at the top of the table and Fabregas admitted it was an important goal. Midfielder Cesc Fabregas runs away in celebration having scored a vital winner in the 88th minute. 'It was vital. We know that we are playing for the

Generated Summary for Row 497:
During the Reclaim Australia rallies on the Easter weekend, anti-racism protester Jacob King was snapped staring into the eyes of a bald man with a swastika tattooed behind his ear. Mr King, from Melbourne, told Daily Mail Australia he had his arms spread out because he thought the man was going to attack him and his fellow protesters at Federation Square. Anti-racism protester Jacob 

TypeError: can only concatenate str (not "float") to str

In [9]:
output_data = pd.DataFrame({'generated_summary': generated_summaries})

# Save the DataFrame to a new CSV file
output_data.to_csv(output_csv_file, index=False)